In [ ]:
import requests

url = "https://yummly2.p.rapidapi.com/feeds/search"

querystring = {"start":"0","maxResult":"30","meatyMax":"0.5","maxTotalTimeInSeconds":"3600","meatyMin":"0.4","FAT_KCALMax":"1000"}

headers = {
    'x-rapidapi-host': "yummly2.p.rapidapi.com",
    'x-rapidapi-key': "0e7d662bd7mshf50a60a4f53eb1cp199dd6jsne50b62cf20ac"
    }

response = requests.request("GET", url, headers=headers, params=querystring).json()

response

In [52]:
import pandas as pd

In [249]:
recipe_dict = {}

for item in response['feed']:
    ingredient_list = []
    for ingredient_group in item['content']['ingredientLines']:
        ingredient_list.append(ingredient_group['ingredient'])
    recipe_dict[item['tracking-id']] = ingredient_list

In [287]:
main_df = pd.DataFrame()

In [288]:
main_df['recipe_id'] = [recipe.replace('recipe/','').split('-')[-1] for recipe in list(recipe_dict.keys())]
main_df['recipe_name'] = [' '.join(recipe.replace('recipe/','').split('-')[:-1]) for recipe in list(recipe_dict.keys())]

In [294]:
main_df['inredients'] = list(recipe_dict.values())

In [295]:
main_df

,recipe_id,recipe_name,inredients
0,9258139,DIY Homemade Taco Seasoning,"[chili powder, ground cumin, salt, ground blac..."
1,1512819,Meatloaf,"[ground beef, finely chopped onion, eggs, plai..."
2,1806994,Glazed Meatloaf,"[ketchup, brown sugar, cider vinegar, hot sauc..."
3,2602005,Easy Tuna Casserole,"[egg noodles, butter, small onion, celery, fro..."
4,9231456,GARLIC BUTTER SEAFOOD BOIL,"[crab legs, shrimp, andouille sausage, corn on..."
5,2502752,Best Salmon Patties,"[pink salmon, eggs, bread crumbs, green onions..."
6,2689522,White Velvet Buttermilk Cake,"[cake flour, granulated sugar, salt, baking po..."
7,9101918,Creamy Chicken Soup,"[oil, unsalted butter, all purpose flour, chic..."
8,2617198,Instant Pot Chipotle Pork Shoulder,"[onions, kosher salt, freshly ground black pep..."
9,2029444,Chicken Pot Pie,"[chicken, sliced carrots, frozen green peas, c..."


In [270]:
exp_string = '-DIY-Homemade-Taco-Seasoning-9258139'
' '.join(exp_string.split('-')[:-1])

' DIY Homemade Taco Seasoning'

In [258]:
for val in recipe_dict.values():
    print(val[0])

chili powder
ground beef
ketchup
egg noodles
crab legs
pink salmon
cake flour
oil
onions
chicken
flour
all purpose flour
all purpose flour
cinnamon
white vinegar
water
Campbell's® Condensed Cream of Chicken Soup
cooking oil spray


In [210]:
my_items = ['potato','eggplant','chicken']

In [246]:
def get_recipe(my_ingredietns):

    query_recipe_dict = recipe_dict

    for recipe in recipe_dict.items():
        query_recipe_dict[[recipe[0]][0]] = round(sum([recipe[1].count(my_item) for my_item in my_items])/len(recipe[1]),3)

    query_recipe_dict = {k: v for k, v in sorted(query_recipe_dict.items(), key=lambda item: item[1], reverse=True)}

    return list(query_recipe_dict)[0].replace('recipe/','')

In [247]:
get_the_recipe(['potato','eggplant','chicken'])

'Chicken-Pot-Pie-2029444'

'-DIY-Homemade-Taco-Seasoning-9258139'

In [146]:
recipe_dict['recipe/-DIY-Homemade-Taco-Seasoning-9258139'] = len(recipe_dict['recipe/-DIY-Homemade-Taco-Seasoning-9258139'])

In [151]:
query_dict = recipe_dict

In [138]:
for recipe in recipe_dict.items():

('recipe/-DIY-Homemade-Taco-Seasoning-9258139', ['chili powder', 'ground cumin', 'salt', 'ground black pepper', 'garlic powder', 'dried oregano', 'paprika', 'onion powder', 'ground beef', 'taco seasoning', 'water', 'flour', 'taco shells', 'taco toppings'])


In [123]:
for recipe_name in recipe_dict.keys():
    print(recipe_name)

recipe/-DIY-Homemade-Taco-Seasoning-9258139
ingedients_num
recipe/Meatloaf-1512819
recipe/Glazed-Meatloaf-1806994
recipe/Easy-Tuna-Casserole-2602005
recipe/GARLIC-BUTTER-SEAFOOD-BOIL-9231456
recipe/Best-Salmon-Patties-2502752
recipe/White-Velvet-Buttermilk-Cake-2689522
recipe/Creamy-Chicken-Soup-9101918
recipe/Instant-Pot-Chipotle-Pork-Shoulder-2617198
recipe/Chicken-Pot-Pie-2029444
recipe/1_-Honey-BBQ-Chicken-Wings-1593238
recipe/Cinnamon-Roll-Waffles-9077581
recipe/Chocolate-Zucchini-Cake-2633214
recipe/Sinful-Apple-Muffins-2211726
recipe/Carne-Asada-Traditional-Taco-2105782
recipe/Avocado-Ranch-Pork-Chops-and-Rice-9343883
recipe/Easy-Chicken-_-Cheese-Enchiladas-9339368
recipe/Chicken-Teriyaki-Noodle-Bowls-9307995


In [119]:
query_dict = {}

for recipe in recipe_dict.items():
    print(recipe)
    #print(recipe[0])
    query_dict['recipe_name'] = recipe[0]
    #print(sum([recipe[1].count(my_item) for my_item in my_items]))
    query_dict['match_rate'] = sum([recipe[1].count(my_item) for my_item in my_items])/recipe[2]
    
    break

('recipe/-DIY-Homemade-Taco-Seasoning-9258139', ['chili powder', 'ground cumin', 'salt', 'ground black pepper', 'garlic powder', 'dried oregano', 'paprika', 'onion powder', 'ground beef', 'taco seasoning', 'water', 'flour', 'taco shells', 'taco toppings'])


In [106]:
query_dict

{'recipe_name': 'recipe/Chicken-Teriyaki-Noodle-Bowls-9307995', 'match': 1}

In [62]:
recipe = ['chili powder', 'ground cumin', 'salt', 'ground black pepper', 'garlic powder', 'dried oregano', 'paprika', 'onion powder', 'ground beef', 'taco seasoning', 'water', 'flour', 'taco shells', 'taco toppings']

In [79]:
sum([recipe.count(my_item) for my_item in my_items])

3